In [ ]:
!nvidia-smi

Fri Dec 18 06:25:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    30W /  70W |   3693MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('data')

Drive already mounted at data; to attempt to forcibly remount, call drive.mount("data", force_remount=True).


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import datetime
import matplotlib.pyplot as plt
from torchsummary import summary
import os
from torchvision.utils import save_image, make_grid

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

class CelebADataset(Dataset):
    def __init__(self, path, transform, factor, device):
        self.path = path
        self.transform = transform
        self.X = []
        self.factor = factor
        for folder in os.listdir(self.path):
            folder = self.path + folder
            for img_path in os.listdir(folder):
                img_path = folder + '/' +  img_path
                img = np.array(Image.open(img_path))
                self.X.append(img)
    
    def __getitem__(self, idx):
        img = Image.fromarray(self.X[idx])
        im = Image.fromarray(self.X[idx])
        imgs_lr = transforms.Resize((37, 37))(img)
        imgs_lr = self.transform(imgs_lr)
        im = transforms.Resize((150, 150))(im)
        imgs_hr = self.transform(im)

        return (imgs_lr.to(device), imgs_hr.to(device))
    
    def __len__(self):
        return len(self.X)

In [ ]:
path = r'/content/data/MyDrive/Intel Image/train/'

if torch.cuda.is_available():
    device = 'cuda' 
else:
    device = 'cpu'

dataset = CelebADataset(path, transform, 4, device)
dataloader = DataLoader(dataset, batch_size= 16, shuffle= True)

In [ ]:
dataset[0][0]

tensor([[[ 0.8431,  0.8510,  0.8588,  ...,  0.8431,  0.8431,  0.8431],
         [ 0.8431,  0.8431,  0.8510,  ...,  0.8667,  0.8588,  0.8588],
         [ 0.8275,  0.8275,  0.8275,  ...,  0.8275,  0.8353,  0.8353],
         ...,
         [-0.9843, -0.8824, -0.8353,  ..., -0.9216, -0.8667, -0.7333],
         [-0.9843, -0.9059, -0.8039,  ..., -0.9843, -0.9922, -0.9843],
         [-0.9922, -0.9765, -0.9137,  ..., -1.0000, -1.0000, -1.0000]],

        [[ 0.7725,  0.7804,  0.7804,  ...,  0.7647,  0.7569,  0.7490],
         [ 0.7098,  0.7176,  0.7255,  ...,  0.7333,  0.7255,  0.7176],
         [ 0.6157,  0.6235,  0.6314,  ...,  0.6235,  0.6235,  0.6314],
         ...,
         [-0.9922, -0.8588, -0.7804,  ..., -0.9137, -0.8588, -0.7490],
         [-0.9765, -0.9137, -0.8118,  ..., -0.9843, -0.9922, -0.9922],
         [-0.9922, -0.9765, -0.9216,  ..., -1.0000, -1.0000, -1.0000]],

        [[ 0.6549,  0.6627,  0.6784,  ...,  0.6549,  0.6471,  0.6392],
         [ 0.5373,  0.5529,  0.5608,  ...,  0

In [ ]:
len(dataset)

8400

In [ ]:
dataset[0][0].shape, dataset[0][1].shape

(torch.Size([3, 37, 37]), torch.Size([3, 150, 150]))

In [ ]:
X1, Y1 = [], []
X2, Y2 = [], []

for item in dataloader:
    im1, im2 = item
    _, c1, x1, y1 = im1.shape
    _, c2, x2, y2 = im2.shape
    X1.append(x1)
    X2.append(x2)
    Y1.append(y1)
    Y2.append(y2)

In [ ]:
sum(X1)/len(X1), sum(X2)/len(X2), sum(Y1)/len(Y1), sum(Y2)/len(Y2)

(37.0, 150.0, 37.0, 150.0)

In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        vgg19_model = torchvision.models.vgg19(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(vgg19_model.features.children())[:18])

    def forward(self, img):
        return self.feature_extractor(img)

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
            nn.PReLU(),
            nn.Conv2d(in_features, in_features, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_features, 0.8),
        )

    def forward(self, x):
        return x + self.conv_block(x)

In [ ]:
class GeneratorResNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, n_residual_blocks=16):
        super(GeneratorResNet, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=9, stride=1, padding=4), 
            nn.PReLU()
            )

        res_blocks = []
        for _ in range(n_residual_blocks):
            res_blocks.append(ResidualBlock(64))
        self.res_blocks = nn.Sequential(*res_blocks)

        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(64, 0.8)
            )

        upsampling = []
        for out_features in range(2):
            upsampling += [
                nn.Conv2d(64, 256, 3, 1, 1),
                nn.BatchNorm2d(256),
                nn.PixelShuffle(upscale_factor=2),
                nn.PReLU(),
            ]
        self.upsampling = nn.Sequential(*upsampling)

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, out_channels, kernel_size=9, stride=1, padding=4), 
            nn.Tanh()
            )

    def forward(self, x):
        out1 = self.conv1(x)
        out = self.res_blocks(out1)
        out2 = self.conv2(out)
        out = torch.add(out1, out2)
        out = self.upsampling(out)
        out = self.conv3(out)
        return out

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, input_shape):
        super(Discriminator, self).__init__()

        self.input_shape = input_shape
        in_channels, in_height, in_width = self.input_shape
        patch_h, patch_w = int(in_height / 2 ** 4), int(in_width / 2 ** 4)
        self.output_shape = (1, patch_h, patch_w)

        def discriminator_block(in_filters, out_filters, first_block=False):
            layers = []
            layers.append(nn.Conv2d(in_filters, out_filters, kernel_size=3, stride=1, padding=1))
            if not first_block:
                layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            layers.append(nn.Conv2d(out_filters, out_filters, kernel_size=3, stride=2, padding=1))
            layers.append(nn.BatchNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        layers = []
        in_filters = in_channels
        for i, out_filters in enumerate([64, 128, 256, 512]):
            layers.extend(discriminator_block(in_filters, out_filters, first_block=(i == 0)))
            in_filters = out_filters
        layers.append(nn.Conv2d(out_filters, 1, kernel_size=3, stride=1, padding=1))
        self.model = nn.Sequential(*layers)

    def forward(self, img):
        return self.model(img)

In [ ]:
feature_ext = FeatureExtractor().float().to(device)
feature_ext.eval()
netG = GeneratorResNet().float().to(device)
netD = Discriminator((3, 150, 150)).float().to(device)

In [ ]:
summary(feature_ext, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           1,792
              ReLU-2         [-1, 64, 150, 150]               0
            Conv2d-3         [-1, 64, 150, 150]          36,928
              ReLU-4         [-1, 64, 150, 150]               0
         MaxPool2d-5           [-1, 64, 75, 75]               0
            Conv2d-6          [-1, 128, 75, 75]          73,856
              ReLU-7          [-1, 128, 75, 75]               0
            Conv2d-8          [-1, 128, 75, 75]         147,584
              ReLU-9          [-1, 128, 75, 75]               0
        MaxPool2d-10          [-1, 128, 37, 37]               0
           Conv2d-11          [-1, 256, 37, 37]         295,168
             ReLU-12          [-1, 256, 37, 37]               0
           Conv2d-13          [-1, 256, 37, 37]         590,080
             ReLU-14          [-1, 256,

In [ ]:
summary(netG, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]          15,616
             PReLU-2         [-1, 64, 150, 150]               1
            Conv2d-3         [-1, 64, 150, 150]          36,928
       BatchNorm2d-4         [-1, 64, 150, 150]             128
             PReLU-5         [-1, 64, 150, 150]               1
            Conv2d-6         [-1, 64, 150, 150]          36,928
       BatchNorm2d-7         [-1, 64, 150, 150]             128
     ResidualBlock-8         [-1, 64, 150, 150]               0
            Conv2d-9         [-1, 64, 150, 150]          36,928
      BatchNorm2d-10         [-1, 64, 150, 150]             128
            PReLU-11         [-1, 64, 150, 150]               1
           Conv2d-12         [-1, 64, 150, 150]          36,928
      BatchNorm2d-13         [-1, 64, 150, 150]             128
    ResidualBlock-14         [-1, 64, 1

In [ ]:
summary(netD, (3, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 150, 150]           1,792
         LeakyReLU-2         [-1, 64, 150, 150]               0
            Conv2d-3           [-1, 64, 75, 75]          36,928
       BatchNorm2d-4           [-1, 64, 75, 75]             128
         LeakyReLU-5           [-1, 64, 75, 75]               0
            Conv2d-6          [-1, 128, 75, 75]          73,856
       BatchNorm2d-7          [-1, 128, 75, 75]             256
         LeakyReLU-8          [-1, 128, 75, 75]               0
            Conv2d-9          [-1, 128, 38, 38]         147,584
      BatchNorm2d-10          [-1, 128, 38, 38]             256
        LeakyReLU-11          [-1, 128, 38, 38]               0
           Conv2d-12          [-1, 256, 38, 38]         295,168
      BatchNorm2d-13          [-1, 256, 38, 38]             512
        LeakyReLU-14          [-1, 256,

In [ ]:
criterion_GAN = nn.MSELoss()
criterion_content = nn.L1Loss()

optimizer_G = torch.optim.Adam(netG.parameters(), lr=1e-3)
optimizer_D = torch.optim.Adam(netD.parameters(), lr=1e-3)

In [ ]:
epochs = 15
i=0
for epoch in range(epochs):
    for batch in dataloader:
        imgs_lr, imgs_hr = batch

        valid = torch.tensor(np.ones((imgs_lr.size(0), 10, 10)), dtype=torch.float32).to(device)
        fake = torch.tensor(np.zeros((imgs_lr.size(0), 10, 10)), dtype=torch.float32).to(device)

        optimizer_G.zero_grad()
        gen_hr = netG(imgs_lr)
        loss_GAN = criterion_GAN(netD(gen_hr), valid)
        gen_features = feature_ext(gen_hr)
        real_features = feature_ext(imgs_hr)
        loss_content = criterion_content(gen_features, real_features.detach())
        loss_G = loss_content + loss_GAN*1e-2
        loss_G.backward()
        optimizer_G.step()

        optimizer_D.zero_grad()
        loss_real = criterion_GAN(netD(imgs_hr), valid)
        loss_fake = criterion_GAN(netD(gen_hr.detach()), fake)
        loss_D = loss_real + loss_fake

        loss_D.backward()
        optimizer_D.step()
    print(f'{i+1} epochs done')
    imgs_lr = nn.functional.interpolate(imgs_lr, scale_factor=4)
    gen_hr = make_grid(gen_hr, nrow=1, normalize=True)
    imgs_lr = make_grid(imgs_lr, nrow=1, normalize=True)
    img_grid = torch.cat((imgs_lr, gen_hr), -1)
    save_image(img_grid, f"/content/data/MyDrive/results/{i+1}.png", normalize=False)
    i+=1

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([16, 10, 10])) that is different to the input size (torch.Size([16, 1, 10, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


1 Epochs done
2 Epochs done
3 Epochs done
4 Epochs done
5 Epochs done
6 Epochs done
7 Epochs done
8 Epochs done
9 Epochs done
10 Epochs done
11 Epochs done
12 Epochs done
13 Epochs done
15 Epochs done
